# Deploying a Hybrid Reasoning LLM

This tutorial walks you through deploying a Hybrid Reasoning LLM using Ray Serve LLM.  

---

## Prerequisites

* Access to GPU compute.
* (Optional) A **Hugging Face token** if using gated models like Meta’s Llama. Store it in `export HF_TOKEN=<YOUR-TOKEN-HERE>`

> Depending on the organization, you can usually request access on the model's Hugging Face page. For example, Meta’s Llama models approval can take anywhere from a few hours to several weeks.

**Dependencies:**  
```bash
pip install "ray[serve,llm]"
```

---

## Distinction with purely reaasoning models

*Hybrid reasoning models* are reasoning-capable models that allow you to toggle the thinking process on and off. This means you can enable structured, step-by-step reasoning when needed but skip it for simpler queries to reduce latency. Pure reasoning models always apply their reasoning behavior, while hybrid models give you fine-grained control over when that reasoning is used.

| **Mode**         | **Core Behavior**                            | **Use Case Examples**                                               | **Limitation**                                    |
| ---------------- | -------------------------------------------- | ------------------------------------------------------------------- | ------------------------------------------------- |
| **Thinking ON**  | Explicit multi-step thinking process | Math, coding, logic puzzles, multi-hop QA, CoT prompting | Slower response time, more tokens used      |
| **Thinking OFF** | Direct answer generation                   | Casual queries, short instructions, single-step answers              | May struggle with complex reasoning or explainability |

> Reasoning often benefit from long context windows (32K—200K tokens), high token throughput, low-temperature decoding (greedy sampling), and strong instruction tuning or scratchpad-style reasoning.

To see an example of deploying a purely reasoning model like *QwQ-32B*, see [Deploying a Reasoning LLM](#)

---

## Enabling or Disabling thinking

---

### When to Enable or Disable Thinking Mode

**When to enable thinking mode:**
- For complex, multi-step tasks that require reasoning, such as math, physics, or logic problems.
- When handling ambiguous queries or situations with incomplete information.
- For planning, workflow orchestration, or when the model needs to act as an "agent" coordinating other tools or models.
- When analyzing intricate data, images, or charts.
- For in-depth code reviews or evaluating outputs from other AI systems (LLM as Judge approach).

**When to disable thinking mode:**
- For simple, well-defined, or routine tasks.
- When low latency and fast responses are the priority.
- When the model is used for repetitive, straightforward steps within a larger automated workflow.

---

### How to enable or disable thinking mode

Toggling the thinking mode varies by model and framework. Always check the model's documentation to see how thinking is structured and controlled.

For example, to [control reasoning in Qwen-3](https://huggingface.co/Qwen/Qwen3-32B#switching-between-thinking-and-non-thinking-mode), you can:
* Add `"/think"` or `"/no_think"` in the prompt
* Or set `enable_thinking` in the request:
  `extra_body={"chat_template_kwargs": {"enable_thinking": ...}}`

See [Sending Requests to an Hybrid Reasoning LLM](#sending-requests-to-an-hybrid-reasoning-llm) for practical examples.

---

## Parsing Reasoning Outputs

In thinking mode, hybrid models often separate *reasoning* from the *final answer* using tags like `<think>...</think>`. Without a proper parser, this reasoning may end up in the `content` field instead of the dedicated `reasoning_content` field.  

To make sure the reasoning output is correctly parsed, configure a `reasoning_parser` in your Ray Serve LLM deployment. This tells vLLM how to isolate the model’s thought process from the rest of the output.
> For example, *Qwen-3* uses the `qwen3` parser. Other models may require different parsers. See the [vLLM docs](https://docs.vllm.ai/en/stable/features/reasoning_outputs.html#supported-models) or your model's documentation to find a supported parser, or [build your own](https://docs.vllm.ai/en/stable/features/reasoning_outputs.html#how-to-support-a-new-reasoning-model) if needed.

```yaml
applications:
- ...
  args:
    llm_configs:
      - model_loading_config:
          model_id: my-qwen-3-32B
          model_source: Qwen/Qwen3-32B
        ...
        engine_kwargs:
          ...
          reasoning_parser: qwen3 # <-- for Qwen-3 models
```

See [Define your deployment](#define-your-deployment) for a complete example.

**Example Response**  
When using a reasoning parser, the response is typically structured like this:

```python
ChatCompletionMessage(
    content="The temperature is...",
    ...,
    reasoning_content="Okay, the user is asking for the temperature today and tomorrow..."
)
```
And you can extract the content and reasoning like this
```python
response = client.chat.completions.create(
  ...
)

print(f"Content: {response.choices[0].message.content}")
print(f"Reasoning: {response.choices[0].message.reasoning_content}")
```

---

## Configure Ray Serve LLM

You can configure your deployment using the Ray Serve LLM Python SDK for fast iteration, or use a Ray Serve config file for better integration and long-term maintainability in your systems.

Make sure to set your Hugging Face token in the config file to access gated models.

We set `tensor_parallel_size= 8` to distribute the model's weights among 4 GPUs in the node. 

::::{tab-set}

:::{tab-item} Python SDK

Ray Serve LLM provides multiple [Python APIs](https://docs.ray.io/en/latest/serve/api/index.html#llm-api) for defining your application. Use [`build_openai_app`](https://docs.ray.io/en/latest/serve/api/doc/ray.serve.llm.build_openai_app.html#ray.serve.llm.build_openai_app) to build a full application from your [`LLMConfig`](https://docs.ray.io/en/latest/serve/api/doc/ray.serve.llm.LLMConfig.html#ray.serve.llm.LLMConfig) object.
```python
from ray import serve
from ray.serve.llm import LLMConfig, build_openai_app
import os

llm_config = LLMConfig(
    model_loading_config=dict(
        model_id="my-qwen-3-32B",
        model_source="Qwen/Qwen3-32B",
    ),
    accelerator_type="A100-40G",
    deployment_config=dict(
        autoscaling_config=dict(
            min_replicas=2, max_replicas=2,
        )
    ),
    engine_kwargs=dict(
        tensor_parallel_size=8,
        max_model_len=32768,
        reasoning_parser='qwen3',
        ### Uncomment if your model is gated and need your Huggingface Token to access it
        #hf_token=os.environ["HF_TOKEN"],
    ),
)
app = build_openai_app({"llm_configs": [llm_config]})

serve.run(app, blocking=True)
```

:::

:::{tab-item} Serve Config (YAML)

In your Ray Serve config file:
```yaml
applications:
- name: my-hybrid-llm-app
  route_prefix: "/"
  import_path: ray.serve.llm:build_openai_app
  args:
    llm_configs:
      - model_loading_config:
          model_id: my-qwen-3-32B
          model_source: Qwen/Qwen3-32B
        accelerator_type: A100-40G
        deployment_config:
          autoscaling_config:
            min_replicas: 2
            max_replicas: 2
        engine_kwargs:
          tensor_parallel_size: 8
          max_model_len: 32768
          reasoning_parser: qwen3
          ### Uncomment if your model is gated and need your Huggingface Token to access it
          #hf_token: <YOUR-TOKEN-HERE>
```

:::

::::

---

## Deploy

There are different ways to deploy your service depending on how you defined it.  

::::{tab-set}

:::{tab-item} From the Python SDK

Follow the instructions at [Configure Ray Serve (Python SDK)](#configure-ray-serve) to define your app in a Python module `serve_my_qwen_3_32B.py`.  
> Make sure your script runs your application with `serve.run(app, blocking=True)`.  

In a terminal, run:  
```bash
python serve_my_qwen_3_32B.py
```

:::

:::{tab-item} From a Serve Config (YAML)  

Follow the instructions at [Configure Ray Serve (Serve Config)](#configure-ray-serve) to define your app with a Ray Serve config file `serve_my_qwen_3_32B.yaml`.  

In a terminal, run:
```bash
serve run serve_my_qwen_3_32B.yaml
```

:::

::::

Deployment typically takes a few minutes as the cluster is provisioned, the vLLM server starts, and the model is downloaded.  

---

## Sending Requests to an Hybrid Reasoning LLM

> Follow the [Deployment instructions](#deploy) to launch your application on your Ray Cluster.

**Retrieve your authentication token and endpoint**  
If running locally, your model will be available at `<YOUR-ENDPOINT-HERE> = "http://localhost:8000"` and you can use a placeholder authentication token: `<YOUR-TOKEN-HERE> = "FAKE_KEY"`
  > **Note:** The OpenAI client requires an `api_key`, but this is not needed for local deployments.  

Otherwise, retrieve both your endpoint and authentication token from your deployment environment’s dashboard or logs.

Use the `model_id` defined in your config (here, `my-qwen-3-32B`) to query your model. See [How to enable or disable thinking mode](#how-to-enable-or-disable-thinking-mode) for more information on how to enable and disable thinking in your hybrid reasoning model. Here are some examples with Qwen-3. 

**Send a request with thinking disabled**  
You can disable thinking in Qwen-3 by either adding a `/no_think` tag in the prompt or by forwarding `enable_thinking: False` to the vLLM inference engine.  

::::{tab-set}

:::{tab-item} Example Curl with `/no_think`
```bash
curl -X POST http://localhost:8000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer FAKE_KEY" \
  -d '{
        "model": "my-qwen-3-32B",
        "messages": [{"role": "user", "content": "What is greater between 7.8 and 7.11 ? /no_think"}]
      }'
```

:::

:::{tab-item} Example Python with `enable_thinking: False`
```python
from urllib.parse import urljoin
from openai import OpenAI

api_key = <YOUR-TOKEN-HERE>
base_url = <YOUR-ENDPOINT-HERE>

client = OpenAI(base_url=urljoin(base_url, "v1"), api_key=api_key)

# Example: Complex query with thinking process
response = client.chat.completions.create(
    model="my-qwen-3-32B",
    messages=[
        {"role": "user", "content": "What's the capital of France ?"}
    ],
    extra_body={"chat_template_kwargs": {"enable_thinking": False}}
)

print(f"Reasoning: \n{response.choices[0].message.reasoning_content}\n\n")
print(f"Answer: \n {response.choices[0].message.content}")
```

:::

::::

**Send a request with thinking enabled**  
You can enable thinking in Qwen-3 by either adding a `/think` tag in the prompt or by forwarding `enable_thinking: True` to the vLLM inference engine.  

::::{tab-set}

:::{tab-item} Example Curl with `/think`
```bash
curl -X POST http://localhost:8000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer FAKE_KEY" \
  -d '{
        "model": "my-qwen-3-32B",
        "messages": [{"role": "user", "content": "What is greater between 7.8 and 7.11 ? /think"}]
      }'
```

:::

:::{tab-item} Example Python with `enable_thinking: True`
```python
from urllib.parse import urljoin
from openai import OpenAI

api_key = <YOUR-TOKEN-HERE>
base_url = <YOUR-ENDPOINT-HERE>

client = OpenAI(base_url=urljoin(base_url, "v1"), api_key=api_key)

# Example: Complex query with thinking process
response = client.chat.completions.create(
    model="my-qwen-3-32B",
    messages=[
        {"role": "user", "content": "What's the capital of France ?"}
    ],
    extra_body={"chat_template_kwargs": {"enable_thinking": True}}
)

print(f"Reasoning: \n{response.choices[0].message.reasoning_content}\n\n")
print(f"Answer: \n {response.choices[0].message.content}")
```

:::

::::

When the reasoning parser is configured correctly, the reasoning output appears in the `reasoning_content` field of the response message. Otherwise, it may be included in the main `content` field, usually wrapped in `<think>...</think>` tags. See [Parsing Reasoning Outputs](#parsing-reasoning-outputs) for details.

---

## Streaming Reasoning Content

In thinking mode, hybrid reasoning models may take longer to begin generating the main content. You can stream their intermediate reasoning output in the same way as the main content.  
```python
from urllib.parse import urljoin
from openai import OpenAI

api_key = <YOUR-TOKEN-HERE>
base_url = <YOUR-ENDPOINT-HERE>

client = OpenAI(base_url=urljoin(base_url, "v1"), api_key=api_key)

# Example: Complex query with thinking process
response = client.chat.completions.create(
    model="my-qwen-3-32B",
    messages=[
        {"role": "user", "content": "I need to plan a trip to Paris from Seattle. Can you help me research flight costs, create an itinerary for 3 days, and suggest restaurants based on my dietary restrictions (vegetarian)?"}
    ],
    extra_body={"chat_template_kwargs": {"enable_thinking": True}},
    stream=True
)

# Stream 
for chunk in response:
    # Stream reasoning content
    if hasattr(chunk.choices[0].delta, "reasoning_content"):
        data_reasoning = chunk.choices[0].delta.reasoning_content
        if data_reasoning:
            print(data_reasoning, end="", flush=True)
    # Later, stream the final answer
    if hasattr(chunk.choices[0].delta, "content"):
        data_content = chunk.choices[0].delta.content
        if data_content:
            print(data_content, end="", flush=True)
```

---

## Summary

In this tutorial, you deployed a reasoning LLM with Ray Serve LLM, from development to production. You learned how to configure Ray Serve LLM with the right reasoning parser, deploy your service on your Ray Cluster, how to send requests, and how to parse reasoning outputs in the response.